In [1]:
#basic
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from sklearn.decomposition import PCA,IncrementalPCA,KernelPCA,FastICA,TruncatedSVD,randomized_svd,NMF
#Pretreatment
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler,QuantileTransformer,RobustScaler,PowerTransformer
from sklearn.feature_selection import VarianceThreshold
#model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import Matern, RationalQuadratic
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

#'wheezy-copper-turtle-magic'가 1인변수만 테스트
train1 = train[train['wheezy-copper-turtle-magic']==1]
test1 = test[test['wheezy-copper-turtle-magic']==1]
train1.reset_index(drop=True,inplace=True)
print(train1.shape)
train1.head()

(510, 258)


,id,muggy-smalt-axolotl-pembus,dorky-peach-sheepdog-ordinal,slimy-seashell-cassowary-goose,snazzy-harlequin-chicken-distraction,frumpy-smalt-mau-ordinal,stealthy-beige-pinscher-golden,chummy-cream-tarantula-entropy,hazy-emerald-cuttlefish-unsorted,nerdy-indigo-wolfhound-sorted,...,wheezy-myrtle-mandrill-entropy,wiggy-lilac-lemming-sorted,gloppy-cerise-snail-contributor,woozy-silver-havanese-gaussian,jumpy-thistle-discus-sorted,muggy-turquoise-donkey-important,blurry-buff-hyena-entropy,bluesy-chocolate-kudu-fepid,gamy-white-monster-expert,target
0,4d1b3f7fff691e39bc3bf0bebaf76eab,-0.652685,-0.320448,-0.990136,0.123433,0.872434,-0.976576,0.986943,-0.654385,0.731564,...,-0.751551,2.214634,0.036100,0.559592,-0.196727,-0.045499,-0.875327,0.949601,0.532729,1
1,98a8077588b7544a04931dad6ba353e9,1.100477,0.146391,0.548201,-0.743063,-0.776455,0.784679,2.884915,4.547408,-1.348984,...,0.127049,-0.591817,-0.694301,-0.356082,-0.689719,-0.164489,0.595378,-0.020808,0.527682,0
2,ffe5a67090871f16d50d157c8218597e,1.603370,1.622184,-1.537069,-0.232444,-0.123593,1.186707,-0.748651,4.826237,-2.024633,...,1.463451,0.308423,-1.266921,-1.025875,-0.894283,-1.959455,-0.355648,0.903022,1.032162,0
3,f2ded623eac31b8dda979e08c1f36d2e,-2.773063,-1.316827,1.019951,-0.155688,0.262493,0.118541,-0.536831,1.759811,1.169555,...,-0.013601,0.455910,0.700794,-1.073916,-0.359614,0.141195,1.320309,-0.146764,-0.568746,1
4,dcb128ab55bbe055bdecc7d6b63774ea,-2.013425,-0.805431,-0.294763,1.629098,-0.013835,0.075623,0.304643,0.027752,-0.959656,...,0.279145,0.675924,-0.492643,0.476203,-0.996044,1.399797,-0.157206,0.015506,1.122774,1


In [4]:
oof1=np.zeros(len(train1))
oof2=np.zeros(len(train1))
oof3=np.zeros(len(train1))
oof4=np.zeros(len(train1))
oof5=np.zeros(len(train1))
oof6=np.zeros(len(train1))
oof7=np.zeros(len(train1))
oof8=np.zeros(len(train1))
oof9=np.zeros(len(train1))

skf = StratifiedKFold(n_splits=5, random_state=42)
vt = VarianceThreshold(threshold=1.5)
poly = PolynomialFeatures(degree=2)
sc = StandardScaler(); rc = RobustScaler()
pca = PCA(svd_solver='full',n_components='mle')

# VarianceThreshold
train2 = vt.fit_transform(train1[cols])
# RobustScaler + VarianceThreshold
train3 = sc.fit_transform(vt.fit_transform(train1[cols]))
# StandardScaler + VarianceThreshold
train4 = rc.fit_transform(vt.fit_transform(train1[cols]))
# StandardScaler + PCA
train5 = sc.fit_transform(pca.fit_transform(train1[cols]))
# RobustScaler + PCA
train6 = rc.fit_transform(pca.fit_transform(train1[cols]))
# PolynomialFeatures + StandardScaler + VarianceThreshold
train7 = poly.fit_transform(sc.fit_transform(vt.fit_transform(train1[cols])))
# PolynomialFeatures + RobustScaler+ VarianceThreshold
train8 = poly.fit_transform(rc.fit_transform(vt.fit_transform(train1[cols])))
# StandardScaler + PolynomialFeatures + VarianceThreshold
train9 = sc.fit_transform(poly.fit_transform(vt.fit_transform(train1[cols])))
# RobustScaler + PolynomialFeatures + VarianceThreshold
train10 = rc.fit_transform(poly.fit_transform(vt.fit_transform(train1[cols])))

In [ ]:
- VarianceThreshold
- RobustScaler + VarianceThreshold + model(NuSVC, QDA, LR, MLP, KNN, SVC, LDA, GPC)
- StandardScaler + VarianceThreshold + model(NuSVC, QDA, LR, MLP, KNN, SVC, LDA, GPC)
- StandardScaler + PCA + model(NuSVC, QDA, LR, MLP, KNN, SVC, LDA, GPC)
- RobustScaler + PCA + model(NuSVC, QDA, LR, MLP, KNN, SVC, LDA, GPC)
- PolynomialFeatures + StandardScaler + VarianceThreshold + model(NuSVC, QDA, LR, MLP, KNN, SVC, LDA, GPC)
- PolynomialFeatures + RobustScaler+ VarianceThreshold + model(NuSVC, QDA, LR, MLP, KNN, SVC, LDA, GPC)
- StandardScaler + PolynomialFeatures + VarianceThreshold + model(NuSVC, QDA, LR, MLP, KNN, SVC, LDA, GPC)
- RobustScaler + PolynomialFeatures + VarianceThreshold + model(NuSVC, QDA, LR, MLP, KNN, SVC, LDA, GPC)

In [6]:
idx1 = train1.index
for train_index, test_index in skf.split(train2, train1['target']):

        clf =NuSVC(probability=True, kernel='poly', degree=4, gamma='auto', random_state=4, nu=0.59, coef0=0.053)
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof1[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        clf.fit(train3[train_index,:],train1.loc[train_index]['target'])
        oof2[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        clf.fit(train4[train_index,:],train1.loc[train_index]['target'])
        oof3[idx1[test_index]] = clf.predict_proba(train4[test_index,:])[:,1]
        clf.fit(train5[train_index,:],train1.loc[train_index]['target'])
        oof4[idx1[test_index]] = clf.predict_proba(train5[test_index,:])[:,1] 
        clf.fit(train6[train_index,:],train1.loc[train_index]['target'])
        oof5[idx1[test_index]] = clf.predict_proba(train6[test_index,:])[:,1] 
        clf.fit(train7[train_index,:],train1.loc[train_index]['target'])
        oof6[idx1[test_index]] = clf.predict_proba(train7[test_index,:])[:,1] 
        clf.fit(train8[train_index,:],train1.loc[train_index]['target'])
        oof7[idx1[test_index]] = clf.predict_proba(train8[test_index,:])[:,1] 
        clf.fit(train9[train_index,:],train1.loc[train_index]['target'])
        oof8[idx1[test_index]] = clf.predict_proba(train9[test_index,:])[:,1] 
        clf.fit(train10[train_index,:],train1.loc[train_index]['target'])
        oof9[idx1[test_index]] = clf.predict_proba(train10[test_index,:])[:,1]         

print('NUSVC1 auc: ', round(roc_auc_score(train1['target'], oof1),5))
print('NUSVC2 auc: ', round(roc_auc_score(train1['target'], oof2),5))
print('NUSVC3 auc: ', round(roc_auc_score(train1['target'], oof3),5))
print('NUSVC4 auc: ', round(roc_auc_score(train1['target'], oof4),5))
print('NUSVC5 auc: ', round(roc_auc_score(train1['target'], oof5),5))
print('NUSVC6 auc: ', round(roc_auc_score(train1['target'], oof6),5))
print('NUSVC7 auc: ', round(roc_auc_score(train1['target'], oof7),5))
print('NUSVC8 auc: ', round(roc_auc_score(train1['target'], oof8),5))
print('NUSVC9 auc: ', round(roc_auc_score(train1['target'], oof9),5))

NUSVC1 auc:  0.93473
NUSVC2 auc:  0.94829
NUSVC3 auc:  0.948
NUSVC4 auc:  0.95185
NUSVC5 auc:  0.95261
NUSVC6 auc:  0.91602
NUSVC7 auc:  0.93979
NUSVC8 auc:  0.92372
NUSVC9 auc:  0.88953


In [7]:
for train_index, test_index in skf.split(train2, train1['target']):

        clf =QuadraticDiscriminantAnalysis(0.1)
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof1[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        clf.fit(train3[train_index,:],train1.loc[train_index]['target'])
        oof2[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        clf.fit(train4[train_index,:],train1.loc[train_index]['target'])
        oof3[idx1[test_index]] = clf.predict_proba(train4[test_index,:])[:,1]
        clf.fit(train5[train_index,:],train1.loc[train_index]['target'])
        oof4[idx1[test_index]] = clf.predict_proba(train5[test_index,:])[:,1] 
        clf.fit(train6[train_index,:],train1.loc[train_index]['target'])
        oof5[idx1[test_index]] = clf.predict_proba(train6[test_index,:])[:,1] 
        clf.fit(train7[train_index,:],train1.loc[train_index]['target'])
        oof6[idx1[test_index]] = clf.predict_proba(train7[test_index,:])[:,1] 
        clf.fit(train8[train_index,:],train1.loc[train_index]['target'])
        oof7[idx1[test_index]] = clf.predict_proba(train8[test_index,:])[:,1] 
        clf.fit(train9[train_index,:],train1.loc[train_index]['target'])
        oof8[idx1[test_index]] = clf.predict_proba(train9[test_index,:])[:,1] 
        clf.fit(train10[train_index,:],train1.loc[train_index]['target'])
        oof9[idx1[test_index]] = clf.predict_proba(train10[test_index,:])[:,1]         

print('QD1 auc: ', round(roc_auc_score(train1['target'], oof1),5))
print('QD2 auc: ', round(roc_auc_score(train1['target'], oof2),5))
print('QD3 auc: ', round(roc_auc_score(train1['target'], oof3),5))
print('QD4 auc: ', round(roc_auc_score(train1['target'], oof4),5))
print('QD5 auc: ', round(roc_auc_score(train1['target'], oof5),5))
print('QD6 auc: ', round(roc_auc_score(train1['target'], oof6),5))
print('QD7 auc: ', round(roc_auc_score(train1['target'], oof7),5))
print('QD8 auc: ', round(roc_auc_score(train1['target'], oof8),5))
print('QD9 auc: ', round(roc_auc_score(train1['target'], oof9),5))

QD1 auc:  0.95848
QD2 auc:  0.95848
QD3 auc:  0.95848
QD4 auc:  0.95642
QD5 auc:  0.95642
QD6 auc:  0.56014
QD7 auc:  0.54065
QD8 auc:  0.52952
QD9 auc:  0.5479


In [8]:
for train_index, test_index in skf.split(train2, train1['target']):

        clf =LogisticRegression(solver='saga',penalty='l2',C=0.01,tol=0.001)
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof1[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        clf.fit(train3[train_index,:],train1.loc[train_index]['target'])
        oof2[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        clf.fit(train4[train_index,:],train1.loc[train_index]['target'])
        oof3[idx1[test_index]] = clf.predict_proba(train4[test_index,:])[:,1]
        clf.fit(train5[train_index,:],train1.loc[train_index]['target'])
        oof4[idx1[test_index]] = clf.predict_proba(train5[test_index,:])[:,1] 
        clf.fit(train6[train_index,:],train1.loc[train_index]['target'])
        oof5[idx1[test_index]] = clf.predict_proba(train6[test_index,:])[:,1] 
        clf.fit(train7[train_index,:],train1.loc[train_index]['target'])
        oof6[idx1[test_index]] = clf.predict_proba(train7[test_index,:])[:,1] 
        clf.fit(train8[train_index,:],train1.loc[train_index]['target'])
        oof7[idx1[test_index]] = clf.predict_proba(train8[test_index,:])[:,1] 
        clf.fit(train9[train_index,:],train1.loc[train_index]['target'])
        oof8[idx1[test_index]] = clf.predict_proba(train9[test_index,:])[:,1] 
        clf.fit(train10[train_index,:],train1.loc[train_index]['target'])
        oof9[idx1[test_index]] = clf.predict_proba(train10[test_index,:])[:,1]         

print('LR auc: ', round(roc_auc_score(train1['target'], oof1),5))
print('LR auc: ', round(roc_auc_score(train1['target'], oof2),5))
print('LR auc: ', round(roc_auc_score(train1['target'], oof3),5))
print('LR auc: ', round(roc_auc_score(train1['target'], oof4),5))
print('LR auc: ', round(roc_auc_score(train1['target'], oof5),5))
print('LR auc: ', round(roc_auc_score(train1['target'], oof6),5))
print('LR auc: ', round(roc_auc_score(train1['target'], oof7),5))
print('LR auc: ', round(roc_auc_score(train1['target'], oof8),5))
print('LR auc: ', round(roc_auc_score(train1['target'], oof9),5))

LR auc:  0.87794
LR auc:  0.87762
LR auc:  0.87505
LR auc:  0.87648
LR auc:  0.87646
LR auc:  0.94006
LR auc:  0.946
LR auc:  0.94067
LR auc:  0.93157


In [9]:
for train_index, test_index in skf.split(train2, train1['target']):

        clf =neural_network.MLPClassifier(random_state=3,  activation='relu',solver='adam' )
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof1[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        clf.fit(train3[train_index,:],train1.loc[train_index]['target'])
        oof2[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        clf.fit(train4[train_index,:],train1.loc[train_index]['target'])
        oof3[idx1[test_index]] = clf.predict_proba(train4[test_index,:])[:,1]
        clf.fit(train5[train_index,:],train1.loc[train_index]['target'])
        oof4[idx1[test_index]] = clf.predict_proba(train5[test_index,:])[:,1] 
        clf.fit(train6[train_index,:],train1.loc[train_index]['target'])
        oof5[idx1[test_index]] = clf.predict_proba(train6[test_index,:])[:,1] 
        clf.fit(train7[train_index,:],train1.loc[train_index]['target'])
        oof6[idx1[test_index]] = clf.predict_proba(train7[test_index,:])[:,1] 
        clf.fit(train8[train_index,:],train1.loc[train_index]['target'])
        oof7[idx1[test_index]] = clf.predict_proba(train8[test_index,:])[:,1] 
        clf.fit(train9[train_index,:],train1.loc[train_index]['target'])
        oof8[idx1[test_index]] = clf.predict_proba(train9[test_index,:])[:,1] 
        clf.fit(train10[train_index,:],train1.loc[train_index]['target'])
        oof9[idx1[test_index]] = clf.predict_proba(train10[test_index,:])[:,1]         

print('MLP1 auc: ', round(roc_auc_score(train1['target'], oof1),5))
print('MLP2 auc: ', round(roc_auc_score(train1['target'], oof2),5))
print('MLP3 auc: ', round(roc_auc_score(train1['target'], oof3),5))
print('MLP4 auc: ', round(roc_auc_score(train1['target'], oof4),5))
print('MLP5 auc: ', round(roc_auc_score(train1['target'], oof5),5))
print('MLP6 auc: ', round(roc_auc_score(train1['target'], oof6),5))
print('MLP7 auc: ', round(roc_auc_score(train1['target'], oof7),5))
print('MLP8 auc: ', round(roc_auc_score(train1['target'], oof8),5))
print('MLP9 auc: ', round(roc_auc_score(train1['target'], oof9),5))

MLP1 auc:  0.91981
MLP2 auc:  0.91967
MLP3 auc:  0.91724
MLP4 auc:  0.91693
MLP5 auc:  0.91439
MLP6 auc:  0.92672
MLP7 auc:  0.9312
MLP8 auc:  0.93116
MLP9 auc:  0.92464


In [10]:
for train_index, test_index in skf.split(train2, train1['target']):

        clf =neighbors.KNeighborsClassifier(n_neighbors=17, p=2.9)
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof1[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        clf.fit(train3[train_index,:],train1.loc[train_index]['target'])
        oof2[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        clf.fit(train4[train_index,:],train1.loc[train_index]['target'])
        oof3[idx1[test_index]] = clf.predict_proba(train4[test_index,:])[:,1]
        clf.fit(train5[train_index,:],train1.loc[train_index]['target'])
        oof4[idx1[test_index]] = clf.predict_proba(train5[test_index,:])[:,1] 
        clf.fit(train6[train_index,:],train1.loc[train_index]['target'])
        oof5[idx1[test_index]] = clf.predict_proba(train6[test_index,:])[:,1] 
        clf.fit(train7[train_index,:],train1.loc[train_index]['target'])
        oof6[idx1[test_index]] = clf.predict_proba(train7[test_index,:])[:,1] 
        clf.fit(train8[train_index,:],train1.loc[train_index]['target'])
        oof7[idx1[test_index]] = clf.predict_proba(train8[test_index,:])[:,1] 
        clf.fit(train9[train_index,:],train1.loc[train_index]['target'])
        oof8[idx1[test_index]] = clf.predict_proba(train9[test_index,:])[:,1] 
        clf.fit(train10[train_index,:],train1.loc[train_index]['target'])
        oof9[idx1[test_index]] = clf.predict_proba(train10[test_index,:])[:,1]         

print('KNN1 auc: ', round(roc_auc_score(train1['target'], oof1),5))
print('KNN2 auc: ', round(roc_auc_score(train1['target'], oof2),5))
print('KNN3 auc: ', round(roc_auc_score(train1['target'], oof3),5))
print('KNN4 auc: ', round(roc_auc_score(train1['target'], oof4),5))
print('KNN5 auc: ', round(roc_auc_score(train1['target'], oof5),5))
print('KNN6 auc: ', round(roc_auc_score(train1['target'], oof6),5))
print('KNN7 auc: ', round(roc_auc_score(train1['target'], oof7),5))
print('KNN8 auc: ', round(roc_auc_score(train1['target'], oof8),5))
print('KNN9 auc: ', round(roc_auc_score(train1['target'], oof9),5))

KNN1 auc:  0.93756
KNN2 auc:  0.9341
KNN3 auc:  0.93343
KNN4 auc:  0.93511
KNN5 auc:  0.92985
KNN6 auc:  0.77233
KNN7 auc:  0.81469
KNN8 auc:  0.78652
KNN9 auc:  0.74774


In [11]:
for train_index, test_index in skf.split(train2, train1['target']):

        clf =SVC(probability=True, kernel='poly', degree=4, gamma='auto', random_state=42)
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof1[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        clf.fit(train3[train_index,:],train1.loc[train_index]['target'])
        oof2[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        clf.fit(train4[train_index,:],train1.loc[train_index]['target'])
        oof3[idx1[test_index]] = clf.predict_proba(train4[test_index,:])[:,1]
        clf.fit(train5[train_index,:],train1.loc[train_index]['target'])
        oof4[idx1[test_index]] = clf.predict_proba(train5[test_index,:])[:,1] 
        clf.fit(train6[train_index,:],train1.loc[train_index]['target'])
        oof5[idx1[test_index]] = clf.predict_proba(train6[test_index,:])[:,1] 
        clf.fit(train7[train_index,:],train1.loc[train_index]['target'])
        oof6[idx1[test_index]] = clf.predict_proba(train7[test_index,:])[:,1] 
        clf.fit(train8[train_index,:],train1.loc[train_index]['target'])
        oof7[idx1[test_index]] = clf.predict_proba(train8[test_index,:])[:,1] 
        clf.fit(train9[train_index,:],train1.loc[train_index]['target'])
        oof8[idx1[test_index]] = clf.predict_proba(train9[test_index,:])[:,1] 
        clf.fit(train10[train_index,:],train1.loc[train_index]['target'])
        oof9[idx1[test_index]] = clf.predict_proba(train10[test_index,:])[:,1]         

print('SVC1 auc: ', round(roc_auc_score(train1['target'], oof1),5))
print('SVC2 auc: ', round(roc_auc_score(train1['target'], oof2),5))
print('SVC3 auc: ', round(roc_auc_score(train1['target'], oof3),5))
print('SVC4 auc: ', round(roc_auc_score(train1['target'], oof4),5))
print('SVC5 auc: ', round(roc_auc_score(train1['target'], oof5),5))
print('SVC6 auc: ', round(roc_auc_score(train1['target'], oof6),5))
print('SVC7 auc: ', round(roc_auc_score(train1['target'], oof7),5))
print('SVC8 auc: ', round(roc_auc_score(train1['target'], oof8),5))
print('SVC9 auc: ', round(roc_auc_score(train1['target'], oof9),5))

SVC1 auc:  0.89894
SVC2 auc:  0.91199
SVC3 auc:  0.88877
SVC4 auc:  0.91922
SVC5 auc:  0.86843
SVC6 auc:  0.91272
SVC7 auc:  0.38114
SVC8 auc:  0.8572
SVC9 auc:  0.88031


In [12]:
for train_index, test_index in skf.split(train2, train1['target']):

        clf =LinearDiscriminantAnalysis(solver='lsqr',shrinkage='auto')
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof1[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        clf.fit(train3[train_index,:],train1.loc[train_index]['target'])
        oof2[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        clf.fit(train4[train_index,:],train1.loc[train_index]['target'])
        oof3[idx1[test_index]] = clf.predict_proba(train4[test_index,:])[:,1]
        clf.fit(train5[train_index,:],train1.loc[train_index]['target'])
        oof4[idx1[test_index]] = clf.predict_proba(train5[test_index,:])[:,1] 
        clf.fit(train6[train_index,:],train1.loc[train_index]['target'])
        oof5[idx1[test_index]] = clf.predict_proba(train6[test_index,:])[:,1] 
        clf.fit(train7[train_index,:],train1.loc[train_index]['target'])
        oof6[idx1[test_index]] = clf.predict_proba(train7[test_index,:])[:,1] 
        clf.fit(train8[train_index,:],train1.loc[train_index]['target'])
        oof7[idx1[test_index]] = clf.predict_proba(train8[test_index,:])[:,1] 
        clf.fit(train9[train_index,:],train1.loc[train_index]['target'])
        oof8[idx1[test_index]] = clf.predict_proba(train9[test_index,:])[:,1] 
        clf.fit(train10[train_index,:],train1.loc[train_index]['target'])
        oof9[idx1[test_index]] = clf.predict_proba(train10[test_index,:])[:,1]         

print('LDA1 auc: ', round(roc_auc_score(train1['target'], oof1),5))
print('LDA2 auc: ', round(roc_auc_score(train1['target'], oof2),5))
print('LDA3 auc: ', round(roc_auc_score(train1['target'], oof3),5))
print('LDA4 auc: ', round(roc_auc_score(train1['target'], oof4),5))
print('LDA5 auc: ', round(roc_auc_score(train1['target'], oof5),5))
print('LDA6 auc: ', round(roc_auc_score(train1['target'], oof6),5))
print('LDA7 auc: ', round(roc_auc_score(train1['target'], oof7),5))
print('LDA8 auc: ', round(roc_auc_score(train1['target'], oof8),5))
print('LDA9 auc: ', round(roc_auc_score(train1['target'], oof9),5))

LDA1 auc:  0.87865
LDA2 auc:  0.87865
LDA3 auc:  0.87865
LDA4 auc:  0.87532
LDA5 auc:  0.87532
LDA6 auc:  0.94364
LDA7 auc:  0.94383
LDA8 auc:  0.94375
LDA9 auc:  0.94375


In [13]:
from sklearn.gaussian_process.kernels import RBF
for train_index, test_index in skf.split(train2, train1['target']):

        clf =GaussianProcessClassifier(2.0 * RBF(1.0))
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof1[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        clf.fit(train3[train_index,:],train1.loc[train_index]['target'])
        oof2[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        clf.fit(train4[train_index,:],train1.loc[train_index]['target'])
        oof3[idx1[test_index]] = clf.predict_proba(train4[test_index,:])[:,1]
        clf.fit(train5[train_index,:],train1.loc[train_index]['target'])
        oof4[idx1[test_index]] = clf.predict_proba(train5[test_index,:])[:,1] 
        clf.fit(train6[train_index,:],train1.loc[train_index]['target'])
        oof5[idx1[test_index]] = clf.predict_proba(train6[test_index,:])[:,1] 
        clf.fit(train7[train_index,:],train1.loc[train_index]['target'])
        oof6[idx1[test_index]] = clf.predict_proba(train7[test_index,:])[:,1] 
        clf.fit(train8[train_index,:],train1.loc[train_index]['target'])
        oof7[idx1[test_index]] = clf.predict_proba(train8[test_index,:])[:,1] 
        clf.fit(train9[train_index,:],train1.loc[train_index]['target'])
        oof8[idx1[test_index]] = clf.predict_proba(train9[test_index,:])[:,1] 
        clf.fit(train10[train_index,:],train1.loc[train_index]['target'])
        oof9[idx1[test_index]] = clf.predict_proba(train10[test_index,:])[:,1]         

print('GPC1 auc: ', round(roc_auc_score(train1['target'], oof1),5))
print('GPC2 auc: ', round(roc_auc_score(train1['target'], oof2),5))
print('GPC3 auc: ', round(roc_auc_score(train1['target'], oof3),5))
print('GPC4 auc: ', round(roc_auc_score(train1['target'], oof4),5))
print('GPC5 auc: ', round(roc_auc_score(train1['target'], oof5),5))
print('GPC6 auc: ', round(roc_auc_score(train1['target'], oof6),5))
print('GPC7 auc: ', round(roc_auc_score(train1['target'], oof7),5))
print('GPC8 auc: ', round(roc_auc_score(train1['target'], oof8),5))
print('GPC9 auc: ', round(roc_auc_score(train1['target'], oof9),5))
#MultinomialNB, GaussianNB, BernoulliNB

GPC1 auc:  0.5
GPC2 auc:  0.92935
GPC3 auc:  0.92883
GPC4 auc:  0.90572
GPC5 auc:  0.91417
GPC6 auc:  0.5
GPC7 auc:  0.5
GPC8 auc:  0.5
GPC9 auc:  0.5
